In [ ]:
# Import python packages
import streamlit as st
import altair as alt
import snowflake.snowpark.functions as F

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
df_support_tickets = session.table('support_tickets')
df_support_tickets

In [ ]:
prompt = """ You are an agent that helps organize the requests that come to your support team

The request category is the reason why the customer reached out. These are the possible types of request categories

Slow performance
Product Info
Account Management
Billing
Technical Support

Try doing it for this request & return only the request category

"""

Use MISTRAL LARGE model

In [ ]:
mistral_large_response_sql= f""" 
        select ticketid, request, trim(snowflake.cortex.complete('mistral-large', concat('{prompt}', request)), '\n') as mistral_large_response
        from support_tickets
"""

df_mistral_large_response = session.sql(mistral_large_response_sql)
df_mistral_large_response

Use a smaller model like MISTRAL-7B to check how are the results bcoz the MISTRAL-LARGE model if run on production on millions of records can be costly

In [ ]:
mistral_7b_response_sql= f""" 
        select ticketid, request, trim(snowflake.cortex.complete('mistral-7b', concat('{prompt}', request)), '\n') as mistral_7b_response
        from support_tickets
"""

df_mistral_7b_response = session.sql(mistral_7b_response_sql)
df_mistral_7b_response

Now lets compare the results of both the models side-by-side

In [ ]:
df_llms = df_mistral_large_response.join(df_mistral_7b_response, 'ticketid')
df_llms

As you can see from above result, the 7B model (i.e. the smaller one) is not producing good results when compared to the mistral-large model.\
So we will now FINE_TUNE the 7B model such that it can produce results as good as mistral-large model

Generate dataset to fine-tune the Mistral-7B

In [ ]:
df_fine_tune = df_mistral_large_response.with_column("prompt", F.concat(F.lit(prompt),F.lit(" "),F.col("request"))).select("ticketid","prompt","mistral_large_response")
df_fine_tune.write.mode('overwrite').save_as_table('support_tickets_finetune')

Finetune Mistral-7b\
Split the data into training set & test set using split function

In [ ]:
train_df, eval_df = session.table("support_tickets_finetune").random_split(weights=[0.8, 0.2], seed=42)
train_df.write.mode('overwrite').save_as_table('support_tickets_train')
eval_df.write.mode('overwrite').save_as_table('support_tickets_eval')

We fine tune the Mistral-7B model through the AI option on the left nav\
Now we use that model

In [ ]:

SELECT SNOWFLAKE.CORTEX.FINETUNE(
  'DESCRIBE',
  'ft_20e3f454-16fa-45ab-84ce-08797c1babd2'
)

In [ ]:
show grants to user AAGARWAL;

use secondary role ALL;

In [ ]:

fine_tuned_model_name = 'RESEARCH.AAGARWAL.ashish_model_finetuned_mistral7b'
sql = f"""select ticketid, request,
trim(snowflake.cortex.complete('{fine_tuned_model_name}',concat('{prompt}',request)),'\n') as fine_tuned_mistral_7b_model_response
from support_tickets"""

df_fine_tuned_mistral_7b_response = session.sql(sql)
df_fine_tuned_mistral_7b_response


In [ ]:
fine_tuned_model_name_5epoch = 'RESEARCH.AAGARWAL.ashish_model_finetuned_mistral7b_5epoch'
sql = f"""select ticketid, request,
trim(snowflake.cortex.complete('{fine_tuned_model_name_5epoch}',concat('{prompt}',request)),'\n') as fine_tuned_mistral_7b_model_response_5epoch
from support_tickets"""

df_fine_tuned_mistral_7b_model_response_5epoch = session.sql(sql)
df_fine_tuned_mistral_7b_model_response_5epoch